In [12]:
import os

import warnings
warnings.filterwarnings("ignore")

with open('openai_api_key.txt', 'r') as f:
    os.environ["OPENAI_API_KEY"] = f.read()

In [13]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI
from langchain.agents import tool
from datetime import date
from langchain import LLMChain
from langchain import PromptTemplate
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from langchain.chains import SequentialChain, RouterChain, ConversationChain, LLMRouterChain, MultiPromptChain, TransformChain
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser


In [14]:
llm = ChatOpenAI(temperature=0)

In [15]:
personalData = """\
    city: London,\
    job-type: [Data Scientist, AI Software Engineer],\
    monthly-earnings: [1500 usd, 8000 usd],\
    skills: [fast learning, reading complicated documentation,\ 
        creating clean data representation, data analysis],\
    interests: [generating ai images, data analysis, large language models],\
    benefits: [flexible hours, remote, health-insurance],\
    technology: [linux -medium, python -advanced, pytorch -advanced, matlabplot -beginner,\
    stable diffusion -beginner, vsc -advanced],\
    languages: [English C1, Polish C1]
    """

In [16]:
queryRank = """Please, process given job offer and at the beginning the answer put a \
    header: "<appropriate: match-level>", where match levels are:\
    [Not appropriate, Acceptable, Ok, Super]. Chosen level shall depend on this how well given job \
    offer matches client job qualifications and preferences.
    job offer: 
    {jobOffer}; \
    client's job qualifications and preferences:
    {personalData};\
    """

promptRank = PromptTemplate(template=queryRank, input_variables=["jobOffer", "personalData"])

chainRank = LLMChain(llm=llm, prompt=promptRank, output_key='Verdict')

In [17]:
jobOffer = "C++ Backend Developer Volue Sp. z o.o. technology image Category: Backend , C++ Mid Must have C++English (B2) Nice to have PythonC#Polish Requirements description C++ programming experience and knowledge of modern C++ Familiarity with Microsoft platform and tools is beneficial Experience working in larger C++ codebases Experience working effectively in a distributed team Some experience with using databases Some knowledge of Python and C# Offer description Volue is a market leader in technologies and services that power the green transition. Around 800 employees work with more than 2 200 customers on energy, power grid, water & infrastructure projects.Our mission is to provide innovative services critical to society, unlocking a cleaner, better and more profitable future. Working towards this goal, Volue has become a leading technology supplier and enabler of the green transition, helping energy companies simplify and optimize everyday operations.We are now looking for Mid/Senior Software Engineer (Mesh Team), who can help us to continuously improve Volue’s ability to provide value to customers in line with the Volue mission. Volue Mesh is a data management system based on object modelling with an integrated time series calculation engine. It is used for more efficient managing of time series data and related information used in different work processes and serves as the core of all future energy production planning and trading solutions.read more Your responsibilities Contribute to all development phases: design, develop, build, deploy and maintain the C++ codebase of Volue Mesh Work closely together with architects, domain experts and backend developers Create and maintain user friendly solutions with clear APIs with efficient resource usage and high performance Work in an international environment, where English is the spoken and written language. The team is distributed in Gdańsk and Trondheim Work in a distributed team with a flexible work-from-home policy Work closely with data intensive applications that are using Mesh and Time Series Management systems to simplify complex workflows show all (7) Job details Online recruitment Recruitment language: Polish&English Start ASAP Permanent contract Remote days: flexible in a week Flexible hours Relocation package: 6000 PLN Perks in the office Free snacks Free coffee Free beverages Gym Canteen Bike parking Playroom Shower No dress code Benefits Sport subscription Life & group insurance Private healthcare English classes Training budget Flat structure Small teams International projects"

In [18]:
result = chainRank({"jobOffer": jobOffer, "personalData": personalData})

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "jobOffer": "C++ Backend Developer Volue Sp. z o.o. technology image Category: Backend , C++ Mid Must have C++English (B2) Nice to have PythonC#Polish Requirements description C++ programming experience and knowledge of modern C++ Familiarity with Microsoft platform and tools is beneficial Experience working in larger C++ codebases Experience working effectively in a distributed team Some experience with using databases Some knowledge of Python and C# Offer description Volue is a market leader in technologies and services that power the green transition. Around 800 employees work with more than 2 200 customers on energy, power grid, water & infrastructure projects.Our mission is to provide innovative services critical to society, unlocking a cleaner, better and more profitable future. Working towards this goal, Volue has become a leading technology supplier and enabler of the green transition, helping energy companies 

In [19]:
print(result)

{'jobOffer': 'C++ Backend Developer Volue Sp. z o.o. technology image Category: Backend , C++ Mid Must have C++English (B2) Nice to have PythonC#Polish Requirements description C++ programming experience and knowledge of modern C++ Familiarity with Microsoft platform and tools is beneficial Experience working in larger C++ codebases Experience working effectively in a distributed team Some experience with using databases Some knowledge of Python and C# Offer description Volue is a market leader in technologies and services that power the green transition. Around 800 employees work with more than 2 200 customers on energy, power grid, water & infrastructure projects.Our mission is to provide innovative services critical to society, unlocking a cleaner, better and more profitable future. Working towards this goal, Volue has become a leading technology supplier and enabler of the green transition, helping energy companies simplify and optimize everyday operations.We are now looking for Mi

In [20]:
superVerdictTemplate = """You are given a job offer and client personal perferences and qualifications. \
    Your task is to briefly, reason why given job offer would be good choice for a client. Be very enthustiastic, \
    try to limit your answer to just two sentences. Write only that reasoning in your answer. \
    Here is job offer: 
    {jobOffer} 
    
    And here are informations about a client:
    {personalData}"""
    
okVerdictTemplate = """You are given a job offer and client personal perferences and qualifications. \
    Your task is to briefly, reason why given job offer would be ok choice for a client. \
    Try to limit your answer to just three sentences. Tell what might not suit a client in the offer. \
    Write only that reasoning in your answer. \
    Here is job offer: 
    {jobOffer} 
    
    And here are informations about a client:
    {personalData}"""
    
acceptableVerdictTemplate = """You are given a job offer and client personal perferences and qualifications. \
    Your task is to briefly, reason why given job offer would be ok choice for a client. \
    Try to limit your answer to just three sentences. Tell that job can be hard to get with the client's qualifications or \
    might just not suit the client's preferences. \
    Write only that reasoning in your answer. \
    Here is job offer: 
    {jobOffer} 
    
    And here are informations about a client:
    {personalData}"""

# def transform_func(inputs: dict) -> dict:
#     return {"description": ""} 

# default_chainsssss = TransformChain(
#     input_variables=["input"],
#     output_variables=["Reasoning"],
#     transform=transform_func
# )

default_chain = ConversationChain(llm=llm, output_key="Reasoning")

prompt_infos = [
    {
        "name": "Acceptable", 
        "description": "Good for describing acceptable job offers", 
        "prompt_template": acceptableVerdictTemplate
    },
    {
        "name": "Ok", 
        "description": "Good for describing Ok job offers", 
        "prompt_template": okVerdictTemplate
    },
    {
        "name": "Super", 
        "description": "Good for describing super job offers", 
        "prompt_template": superVerdictTemplate
    },
]

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=['personalData', 'jobOffer'])
    chain = LLMChain(llm=llm, prompt=prompt, output_key="Reasoning")
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
 
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{Verdict}}

<< OUTPUT (remember to include the ```json)>>""" 
  
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)  
        
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["Verdict"],
    output_parser=RouterOutputParser(),
)

routerChainDesc = LLMRouterChain.from_llm(llm, router_prompt)

chainDescription = MultiPromptChain(router_chain=routerChainDesc, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain)

ValidationError: 1 validation error for PromptTemplate
__root__
  Invalid prompt schema; check for mismatched or missing input parameters. {'jobOffer', 'personalData'} (type=value_error)

In [ ]:
overall_chain = SequentialChain(
    chains=[chainRank, chainDescription],
    input_variables=["jobOffer", "personalData"],
    output_variables=["Verdict", "text"],
    verbose=True
)

In [ ]:
import langchain


langchain.debug = True
result = overall_chain({"jobOffer": jobOffer, "personalData": personalData})
print(result)

[chain/start] [1:chain:SequentialChain] Entering Chain run with input:
{
  "jobOffer": "C++ Backend Developer Volue Sp. z o.o. technology image Category: Backend , C++ Mid Must have C++English (B2) Nice to have PythonC#Polish Requirements description C++ programming experience and knowledge of modern C++ Familiarity with Microsoft platform and tools is beneficial Experience working in larger C++ codebases Experience working effectively in a distributed team Some experience with using databases Some knowledge of Python and C# Offer description Volue is a market leader in technologies and services that power the green transition. Around 800 employees work with more than 2 200 customers on energy, power grid, water & infrastructure projects.Our mission is to provide innovative services critical to society, unlocking a cleaner, better and more profitable future. Working towards this goal, Volue has become a leading technology supplier and enabler of the green transition, helping energy com

ValueError: Missing some input keys: {'jobOffer', 'personalData'}